In [1]:
# Thêm autoreload vào để tự động reload lại module nếu có thay đổi code trong module
%load_ext autoreload
%autoreload 2

import mediapipe as mp
import math
import numpy as np
import pandas as pd
import cv2
import copy
import warnings
warnings.filterwarnings('ignore')

import os, sys
sys.path.append(os.path.abspath(".."))
from utils.common import load_model, get_color_for_landmarks

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Thực hiện việc dự đoán với các model Scikit learn có độ chính xác cao nhất

In [2]:
# Load model
# RF_model.pkl là sau điều chỉnh tham số
RF_model = load_model('./best_models/RF_model.pkl')

# Load input scaler
input_scaler = load_model("./best_models/input_scaler.pkl")

In [3]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX"
]

# Tạo các cột cho dữ liệu đầu vào
HEADERS = ["label"]
for landmark in IMPORTANT_LMS:
    for dim in ['x', 'y', 'z']:
        HEADERS.append(f"{landmark.lower()}_{dim}")

In [4]:
def extract_and_recalculate_landmarks(pose_landmarks):
    """
    Tịnh tiến thân người vào giữa bức hình, đồng thời dời lại trục toạ độ
    """
    hip_center_x = float((pose_landmarks[23].x + pose_landmarks[24].x) / 2)
    hip_center_y = float((pose_landmarks[23].y + pose_landmarks[24].y) / 2)

    new_center = (0.5, 0.5)
    delta_x = new_center[0] - hip_center_x
    delta_y = new_center[1] - hip_center_y

    data = []
    for landmark in IMPORTANT_LMS:
        # Lấy ra id của key point trên cơ thể người
        key_point_id = mp_pose.PoseLandmark[landmark].value

        key_point = pose_landmarks[key_point_id]
        key_point.x += delta_x - 0.5
        key_point.y += delta_y -0.5
        data.append([key_point.x, key_point.y, key_point.z])

    return np.array(data).flatten().tolist()


def extract_important_key_points(results) -> list:
    key_points = []

    for lm in IMPORTANT_LMS:
        # Lấy ra id của key point trên cơ thể người
        key_point_id = mp_pose.PoseLandmark[lm].value
        key_point = results.pose_landmarks.landmark[key_point_id]
        key_points.append([key_point.x, key_point.y, key_point.z])

    return np.array(key_points).flatten().tolist()

def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [5]:
def get_class(encode_label: float):
    return {
        0: "C",
        1: "W"
    }.get(encode_label, "Unknown")

### Dùng phương pháp hình học để xác định lỗi sai

In [6]:
# xác định chiều dài và chiều rộng của bức ảnh
def get_image_size(image):
    return image.shape[1], image.shape[0]

In [7]:
def calculate_angle(a, b, c, size_of_image):
    # Lấy tọa độ của 3 điểm
    a = (a[0] * size_of_image[0], a[1] * size_of_image[1])
    b = (b[0] * size_of_image[0], b[1] * size_of_image[1])
    c = (c[0] * size_of_image[0], c[1] * size_of_image[1])

    # Tính góc giữa 3 điểm
    angle = math.degrees(math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    
    # Chuyển góc về khoảng từ 0 đến 360 độ
    if angle < 0:
        angle += 360
    return angle

In [8]:
def define_error(key_points, image_size):
    error = []
    angle = 180

    if (key_points.nose_x[0] > 0):
        angle = calculate_angle((key_points.right_shoulder_x[0], key_points.right_shoulder_y[0]), 
                        (key_points.right_hip_x[0], key_points.right_hip_y[0]), 
                        (key_points.right_ankle_x[0], key_points.right_ankle_y[0]), 
                        image_size)
    else:
        angle = calculate_angle((key_points.left_shoulder_x[0], key_points.left_shoulder_y[0]), 
                        (key_points.left_hip_x[0], key_points.left_hip_y[0]), 
                        (key_points.left_ankle_x[0], key_points.left_ankle_y[0]), 
                        image_size)

    if angle < 170:
        error.append("high hip")
    elif angle > 190:
        error.append("low hip")

    angle = 90
    if (key_points.nose_x[0] > 0):
        angle = calculate_angle((key_points.right_shoulder_x[0], key_points.right_shoulder_y[0]),
                            (key_points.right_elbow_x[0], key_points.right_elbow_y[0]),
                            (key_points.right_wrist_x[0], key_points.right_wrist_y[0]),
                            image_size)
    else:
        angle = calculate_angle((key_points.left_shoulder_x[0], key_points.left_shoulder_y[0]),
                            (key_points.left_elbow_x[0], key_points.left_elbow_y[0]),
                            (key_points.left_wrist_x[0], key_points.left_wrist_y[0]),
                            image_size)
    
    if angle < 75:
        error.append("elbow to front")
    elif angle > 105:
        error.append("elbow to after")

    if error == []:
        return "Unknown"
    else:
        return ", ".join(error)

In [9]:
def plank_detection(video_path, prediction_probability_threshold = 0.5):
    cap = cv2.VideoCapture(video_path if video_path else 0)
    current_class = "Unknown"

    # Tạo cửa sổ hiển thị
    #cv2.namedWindow("CV2", cv2.WINDOW_NORMAL)
    #cv2.resizeWindow("CV2", 1280, 720)  # Kích thước mặc định cho cửa sổ

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()

            if not ret:
                print("Ignoring empty camera frame.")
                break
            
            # resize frame để tăng tốc độ xử lý
            image = rescale_frame(image, percent=50)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if not results.pose_landmarks:
                print("No human found")
                continue

            initial_pose_landmarks = copy.deepcopy(results.pose_landmarks)
            image.flags.writeable = True

            # Cần khôi phục lại màu gốc của ảnh
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Get landmarks
            try:
                key_points = extract_and_recalculate_landmarks(results.pose_landmarks.landmark)
                X = pd.DataFrame([key_points], columns=HEADERS[1:])
                error = define_error(X, get_image_size(image))
                X = input_scaler.transform(X)

                predicted_class = RF_model.predict(X)[0]
                predicted_class = get_class(RF_model.predict(X)[0])
                prediction_probability_max = RF_model.predict_proba(X)[0].max()

                if prediction_probability_max >= prediction_probability_threshold:
                    current_class = predicted_class
                else:
                    current_class = "Unknown"

                colors = get_color_for_landmarks(current_class)
                mp_drawing.draw_landmarks(
                        image,
                        initial_pose_landmarks,
                        mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=colors[0], thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=colors[1], thickness=2, circle_radius=1),
                    )

                cv2.rectangle(image, (0, 0), (750, 60), (245, 117, 16), -1)

                # Display error
                cv2.putText(image, "ERROR", (180, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                if (current_class == "Unknown"):
                    cv2.putText(image, "Unknown", (180, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                elif (current_class == "W"):
                    cv2.putText(image, error, (180, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(image, "OK", (180, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display class
                cv2.putText(image, "CLASS", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_class, (110, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display probability
                cv2.putText(image, "PROB", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(prediction_probability_max, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("CV2", image)
            
            # Nhấn q để thoát
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [10]:
plank_detection("3.mp4")